In [ ]:
import numpy as np
import pandas as pd
import snowflake.connector as snow
from snowflake.connector import pandas_tools
import matplotlib.pyplot as plt
from itertools import product
import seaborn as sns

In [ ]:
# ACS Variables List to Iterate
acs_vars_list = ['CHURNED_SUB_FLAG', 'ACS_TOTAL_POP',
       'ACS_F_TOTAL_POP_PROP', 'ACS_M_TOTAL_POP_PROP', 'ACS_OVER_65_PROP',
       'ACS_60_64_PROP', 'ACS_50_59_PROP', 'ACS_40_49_PROP', 'ACS_30_39_PROP',
       'ACS_22_29_PROP', 'ACS_18_21_PROP', 'ACS_10_17_PROP',
       'ACS_UNDER_10_PROP', 'TOTAL_POP_MEDIAN_AGE', 'ACS_HP_PROP',
       'ACS_NOT_HP_ASIAN_ALONE_PROP', 'ACS_NOT_HP_AFRICAN_AMERICAN_ALONE_PROP',
       'ACS_NOT_HP_WHITE_ALONE_PROP', 'ACS_NOT_HP_OTHER_POP',
       'ACS_HH_INCOME_200K_MORE_PROP', 'ACS_HH_INCOME_150K_199K_PROP',
       'ACS_HH_INCOME_125K_149K_PROP', 'ACS_HH_INCOME_100K_124K_PROP',
       'ACS_HH_INCOME_75K_99K_PROP', 'ACS_HH_INCOME_60K_74K_PROP',
       'ACS_HH_INCOME_50K_59K_PROP', 'ACS_HH_INCOME_45K_49K_PROP',
       'ACS_HH_INCOME_40K_44K_PROP', 'ACS_HH_INCOME_35K_39K_PROP',
       'ACS_HH_INCOME_30K_34K_PROP', 'ACS_HH_INCOME_25K_29K_PROP',
       'ACS_HH_INCOME_20K_24K_PROP', 'ACS_HH_INCOME_15K_19K_PROP',
       'ACS_HH_INCOME_10K_14K_PROP', 'ACS_HH_INCOME_10K_LESS_PROP',
       'ACS_MEDIAN_EARNINGS_OTHER_DOLLARS',
       'ACS_MEDIAN_EARNINGS_FULLTIME_DOLLARS',
       'ACS_MEDIAN_EARNINGS_OVERALL_DOLLARS', 'ACS_AGG_HH_EARNINGS_PROP',
       'ACS_NO_EARNING_HH_PROP', 'ACS_W_EARNING_HH_PROP', 'ACS_HH_TOTAL',
       'ACS_HH_5_OR_MORE_PROP', 'ACS_HH_4_PROP', 'ACS_HH_3_PROP',
       'ACS_HH_2_PROP', 'ACS_HH_1_PROP', 'ACS_TOTAL_WORKERS_OVER_16', 'ACS_TOTAL_WORKERS_PROP',
       'ACS_WORK_COMMUTE_LESS_15_MIN_PROP', 'ACS_WORK_COMMUTE_15_29_MIN_PROP',
       'ACS_WORK_COMMUTE_30_59_MIN_PROP', 'ACS_WORK_COMMUTE_OVER_60_MIN_PROP']

acs_vars_nochurnvar_list = acs_vars_list[1:]



In [ ]:
# Connection Details
con = snow.connect(
    user="DESENSITIZED",
    server="DESENSITIZED",
    database="DESENSITIZED",
    warehouse="DESENSITIZED",
    authenticator="externalbrowser",
    account="DESENSITIZED"
)

In [ ]:
# Get Master Joined Table
# Age No Bucket
# Create a cursor object.
cur = con.cursor()

# Execute a statement that will generate a result set.
sub_data_sql = '''
select SUB_BILLING_ID  
,SUB_STATUS
,CHURNED_SUB_FLAG
, ACS_TOTAL_POP
, ACS_F_TOTAL_POP_PROP
, ACS_M_TOTAL_POP_PROP
, ACS_OVER_65_PROP
, ACS_60_64_PROP
, ACS_50_59_PROP
, ACS_40_49_PROP
, ACS_30_39_PROP
, ACS_22_29_PROP
, ACS_18_21_PROP
, ACS_10_17_PROP
, ACS_UNDER_10_PROP
, TOTAL_POP_MEDIAN_AGE
, ACS_HP_PROP
, ACS_NOT_HP_ASIAN_ALONE_PROP
, ACS_NOT_HP_AFRICAN_AMERICAN_ALONE_PROP
, ACS_NOT_HP_WHITE_ALONE_PROP
, (1 - ACS_NOT_HP_ASIAN_ALONE_PROP - ACS_NOT_HP_AFRICAN_AMERICAN_ALONE_PROP - ACS_NOT_HP_WHITE_ALONE_PROP) as ACS_NOT_HP_OTHER_POP
, ACS_HH_INCOME_200K_MORE_PROP
, ACS_HH_INCOME_150K_199K_PROP
, ACS_HH_INCOME_125K_149K_PROP
, ACS_HH_INCOME_100K_124K_PROP
, ACS_HH_INCOME_75K_99K_PROP
, ACS_HH_INCOME_60K_74K_PROP
, ACS_HH_INCOME_50K_59K_PROP
, ACS_HH_INCOME_45K_49K_PROP
, ACS_HH_INCOME_40K_44K_PROP
, ACS_HH_INCOME_35K_39K_PROP
, ACS_HH_INCOME_30K_34K_PROP
, ACS_HH_INCOME_25K_29K_PROP
, ACS_HH_INCOME_20K_24K_PROP
, ACS_HH_INCOME_15K_19K_PROP
, ACS_HH_INCOME_10K_14K_PROP
, ACS_HH_INCOME_10K_LESS_PROP
, ACS_MEDIAN_EARNINGS_OTHER_DOLLARS
, ACS_MEDIAN_EARNINGS_FULLTIME_DOLLARS
, ACS_MEDIAN_EARNINGS_OVERALL_DOLLARS
, ACS_AGG_HH_EARNINGS_PROP
, ACS_NO_EARNING_HH_PROP
, ACS_W_EARNING_HH_PROP
, ACS_HH_TOTAL
, ACS_HH_5_OR_MORE_PROP
, ACS_HH_4_PROP
, ACS_HH_3_PROP
, ACS_HH_2_PROP
, ACS_HH_1_PROP
, ACS_TOTAL_WORKERS_OVER_16
, round(div0null(ACS_TOTAL_WORKERS_OVER_16, ACS_TOTAL_POP), 2) as ACS_TOTAL_WORKERS_PROP
, ACS_WORK_COMMUTE_LESS_15_MIN_PROP
, ACS_WORK_COMMUTE_15_29_MIN_PROP
, ACS_WORK_COMMUTE_30_59_MIN_PROP
, ACS_WORK_COMMUTE_OVER_60_MIN_PROP
, SNAPSHOT_DATE
from DESENSITIZED;
'''
cur.execute(sub_data_sql)

# Fetch the result set from the cursor and deliver it as the pandas DataFrame.
sub_data_df = cur.fetch_pandas_all()

In [ ]:
sub_data_df

In [ ]:
sub_vars_df = sub_data_df[acs_vars_nochurnvar_list]

In [ ]:
sub_vars_df

Missing Data Analysis

In [ ]:
# Calculate the proportion of missing data for each individual column
missing_data_proportion = sub_vars_df.isnull().mean()

# Sort the proportion of missing data in descending order
missing_data_proportion = missing_data_proportion.sort_values(ascending=False)

# Plot the proportion of missing data
plt.figure(figsize=(24,6))
# Plot the proportion of missing data
ax = missing_data_proportion.plot(kind='bar')
plt.title('Proportion of Missing Data for Each Column')
plt.xlabel('Columns')
plt.ylabel('Proportion of Missing Data')

# Display the values of each bar above the bar
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')


plt.show()

In [ ]:
# How many subs out of total that have the "core" columns

acs_core_nochurn_cols = ['ACS_TOTAL_POP',
       'ACS_F_TOTAL_POP_PROP', 'ACS_M_TOTAL_POP_PROP', 'ACS_OVER_65_PROP',
       'ACS_60_64_PROP', 'ACS_50_59_PROP', 'ACS_40_49_PROP', 'ACS_30_39_PROP',
       'ACS_22_29_PROP', 'ACS_18_21_PROP', 'ACS_10_17_PROP',
       'ACS_UNDER_10_PROP', 'TOTAL_POP_MEDIAN_AGE', 'ACS_HP_PROP',
       'ACS_NOT_HP_ASIAN_ALONE_PROP', 'ACS_NOT_HP_AFRICAN_AMERICAN_ALONE_PROP',
       'ACS_NOT_HP_WHITE_ALONE_PROP', 'ACS_NOT_HP_OTHER_POP',
       'ACS_HH_INCOME_200K_MORE_PROP', 'ACS_HH_INCOME_150K_199K_PROP',
       'ACS_HH_INCOME_125K_149K_PROP', 'ACS_HH_INCOME_100K_124K_PROP',
       'ACS_HH_INCOME_75K_99K_PROP', 'ACS_HH_INCOME_60K_74K_PROP',
       'ACS_HH_INCOME_50K_59K_PROP', 'ACS_HH_INCOME_45K_49K_PROP',
       'ACS_HH_INCOME_40K_44K_PROP', 'ACS_HH_INCOME_35K_39K_PROP',
       'ACS_HH_INCOME_30K_34K_PROP', 'ACS_HH_INCOME_25K_29K_PROP',
       'ACS_HH_INCOME_20K_24K_PROP', 'ACS_HH_INCOME_15K_19K_PROP',
       'ACS_HH_INCOME_10K_14K_PROP', 'ACS_HH_INCOME_10K_LESS_PROP',
       'ACS_NO_EARNING_HH_PROP', 'ACS_W_EARNING_HH_PROP', 'ACS_HH_TOTAL',
       'ACS_HH_5_OR_MORE_PROP', 'ACS_HH_4_PROP', 'ACS_HH_3_PROP',
       'ACS_HH_2_PROP', 'ACS_HH_1_PROP', 'ACS_TOTAL_WORKERS_OVER_16', 'ACS_TOTAL_WORKERS_PROP',
       'ACS_WORK_COMMUTE_LESS_15_MIN_PROP', 'ACS_WORK_COMMUTE_15_29_MIN_PROP',
       'ACS_WORK_COMMUTE_30_59_MIN_PROP', 'ACS_WORK_COMMUTE_OVER_60_MIN_PROP']

# Calculate the proportion of rows where there are only missing values in core columns
missing_all_core_only = sub_vars_df[acs_core_nochurn_cols].isnull().all(axis=1).mean()

print("The proportion of rows with missing values only in columns B and C is " + str(round(missing_all_core_only, 3)) + ". With a Total Sub Rows of " + str(sub_vars_df.shape[0]) + ", this makes a total of " + str(int(sub_vars_df.shape[0]*(1-missing_all_core_only))) + " rows with most of the ACS data.")

Distribution/Churn Analysis

In [ ]:
# Grab only all non-null columns from the ACS Core Cols
ACS_nonnull_df = sub_data_df[acs_vars_list].dropna(subset=acs_core_nochurn_cols)

In [ ]:
ACS_nonnull_df.drop(columns = 'ACS_TOTAL_WORKERS_OVER_16', inplace = True)

In [ ]:
ACS_nonnull_df

In [ ]:
[0,500,1000,1500,2000,2500,3000,
3500,4000,4500,5000,20000]

In [ ]:
# Bins Definition

# ACS_TOTAL_POP
TP_scheme = [0,500,1000,1500,2000,2500,3000,
3500,4000,4500,5000,20000]
TP_labels = [f'{i}-{i+500}' for i in range(0, 5000, 500)] + ['5000-20000']

# Gender
gender_scheme = [0] + [i/100 for i in range(20, 85, 5)] + [1.0]
gender_labels = ['0-20%'] + [f'{i}-{i+5}%' for i in range(20, 80, 5)] + ['80-100%']

eth_scheme = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
eth_labels = ['0-10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '70-80%', '80-90%', '90-100%']

age_scheme = [i/100 for i in range(0, 55, 5)] + [1.0]
age_labels = [f'{i}-{i+5}%' for i in range(0, 50, 5)] + ['50-100%']

median_age_scheme = [0] + list(range(15, 70, 5)) + [100]
median_age_labels = ['0-15'] + [f'{i}-{i+5}' for i in range(15, 65, 5)] + ['65-100']

hh_total_scheme = list(range(0, 2100, 100)) + [8000]
hh_total_labels = [f'{i}-{i+100}' for i in range(0, 2000, 100)] + ['2000-7500']

hh_earning_scheme = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
hh_earning_labels = ['0-10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '70-80%', '80-90%', '90-100%']

hh_income_scheme = [i/100 for i in range(0, 55, 5)] + [1.0]
hh_income_labels = [f'{i}-{i+5}%' for i in range(0, 50, 5)] + ['50-100%']

hh_size_scheme = [i/100 for i in range(0, 55, 5)] + [1.0]
hh_size_labels = [f'{i}-{i+5}%' for i in range(0, 50, 5)] + ['50-100%']

total_worker_scheme = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
total_worker_labels = ['0-10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '70-80%', '80-90%', '90-100%']

commute_time_scheme = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
commute_time_labels = ['0-10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '70-80%', '80-90%', '90-100%']

# Define Function to use for all Categories of Interest
def churn_percentage(group):
    return round(group['CHURNED_SUB_FLAG'].mean() * 100, 2)

def churn_group_count(group):
    return group['CHURNED_SUB_FLAG'].count()

def acs_summary_stats(df, data_cols, bin_scheme, label_scheme):
    subset_cols = ["CHURNED_SUB_FLAG"] + data_cols
    subset_df = df[subset_cols]
    print(subset_cols)
    
    # Apply binning and groupby for each Ethnicity column
    for col in data_cols:
        subset_df[f'{col}_bin'] = pd.cut(subset_df[col], bins=bin_scheme, labels=label_scheme, include_lowest = True)
        summary_1 = subset_df.groupby(f'{col}_bin').apply(churn_percentage).reset_index(name=f'ChurnPercentage')
        summary_2 = subset_df.groupby(f'{col}_bin').apply(churn_group_count).reset_index(name=f'GroupSize')
        print(summary_1)
        print(summary_2)
        summary_1.to_csv(f'C:/Users/davidl/OneDrive - ULTRA MOBILE/Desktop/dli_code/raw_zone/{col}_perc.csv', header=True)
        summary_2.to_csv(f'C:/Users/davidl/OneDrive - ULTRA MOBILE/Desktop/dli_code/raw_zone/{col}_count.csv', header=True)

Total Pop

In [ ]:
acs_summary_stats(ACS_nonnull_df, ['ACS_TOTAL_POP'], TP_scheme, TP_labels)

Gender

In [ ]:
acs_summary_stats(ACS_nonnull_df, ['ACS_F_TOTAL_POP_PROP', 'ACS_M_TOTAL_POP_PROP'], gender_scheme, gender_labels)

Ethnicity

In [ ]:
acs_summary_stats(ACS_nonnull_df, ['ACS_HP_PROP','ACS_NOT_HP_ASIAN_ALONE_PROP', 'ACS_NOT_HP_AFRICAN_AMERICAN_ALONE_PROP','ACS_NOT_HP_WHITE_ALONE_PROP', 'ACS_NOT_HP_OTHER_POP'], eth_scheme, eth_labels)

Age

In [ ]:
acs_summary_stats(ACS_nonnull_df, ['ACS_OVER_65_PROP',
       'ACS_60_64_PROP', 'ACS_50_59_PROP', 'ACS_40_49_PROP', 'ACS_30_39_PROP',
       'ACS_22_29_PROP', 'ACS_18_21_PROP', 'ACS_10_17_PROP',
       'ACS_UNDER_10_PROP'], age_scheme, age_labels)

Median Age

In [ ]:
acs_summary_stats(ACS_nonnull_df, ['TOTAL_POP_MEDIAN_AGE'], median_age_scheme, median_age_labels)

Proportion HH Earning

In [ ]:
acs_summary_stats(ACS_nonnull_df, ['ACS_NO_EARNING_HH_PROP', 'ACS_W_EARNING_HH_PROP'], hh_earning_scheme, hh_earning_labels)

Total Workers

In [ ]:
acs_summary_stats(ACS_nonnull_df, ['ACS_TOTAL_WORKERS_PROP'], total_worker_scheme, total_worker_labels)

HH Total

In [ ]:
acs_summary_stats(ACS_nonnull_df, ['ACS_HH_TOTAL'], hh_total_scheme, hh_total_labels)

HH Income

In [ ]:
acs_summary_stats(ACS_nonnull_df, ['ACS_HH_INCOME_200K_MORE_PROP', 'ACS_HH_INCOME_150K_199K_PROP',
       'ACS_HH_INCOME_125K_149K_PROP', 'ACS_HH_INCOME_100K_124K_PROP',
       'ACS_HH_INCOME_75K_99K_PROP', 'ACS_HH_INCOME_60K_74K_PROP',
       'ACS_HH_INCOME_50K_59K_PROP', 'ACS_HH_INCOME_45K_49K_PROP',
       'ACS_HH_INCOME_40K_44K_PROP', 'ACS_HH_INCOME_35K_39K_PROP',
       'ACS_HH_INCOME_30K_34K_PROP', 'ACS_HH_INCOME_25K_29K_PROP',
       'ACS_HH_INCOME_20K_24K_PROP', 'ACS_HH_INCOME_15K_19K_PROP',
       'ACS_HH_INCOME_10K_14K_PROP', 'ACS_HH_INCOME_10K_LESS_PROP'], hh_income_scheme, hh_income_labels)

HH Size

In [ ]:
acs_summary_stats(ACS_nonnull_df, ['ACS_HH_5_OR_MORE_PROP', 'ACS_HH_4_PROP', 'ACS_HH_3_PROP',
       'ACS_HH_2_PROP', 'ACS_HH_1_PROP'], hh_size_scheme, hh_size_labels)

Commute Time

In [ ]:
acs_summary_stats(ACS_nonnull_df, ['ACS_WORK_COMMUTE_LESS_15_MIN_PROP', 'ACS_WORK_COMMUTE_15_29_MIN_PROP',
       'ACS_WORK_COMMUTE_30_59_MIN_PROP', 'ACS_WORK_COMMUTE_OVER_60_MIN_PROP'], commute_time_scheme, commute_time_labels)

Combine into One Excel File

import os

# Directory containing the CSV files
directory = 'C:/Users/davidl/OneDrive - ULTRA MOBILE/Desktop/dli_code'

# Create a Pandas Excel writer using openpyxl as the engine
with pd.ExcelWriter('combined_churn_analysis.xlsx', engine='openpyxl') as writer:
    # Loop through all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            # Read each CSV file
            df = pd.read_csv(os.path.join(directory, filename))
            # Write the DataFrame to a specific sheet in the Excel file
            sheet_name = os.path.splitext(filename)[0]  # Use the filename (without extension) as the sheet name
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            # Delete the CSV file after writing its data to the Excel file
            os.remove(os.path.join(directory, filename))

print('All CSV files have been combined into one Excel file and the original CSV files have been deleted.')